# Motif Discovery Visualization

## Imports

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
from pathlib import Path
from ipywidgets import interact_manual

from runBenchmark import getBenchmarkFiles, algorithms
from plot import plotMotif

## Find benchmark files

You can generate the benchmark using the _generateBenchmark.py_ script. The benchmark path can be modified below.

In [ ]:
benchmarkPath = Path("./benchmark")  # change path to benchmark here

exists = benchmarkPath.exists()
print("Benchmark Path exists:", exists)
filePaths = {}
count = 0
if exists:
    for files in getBenchmarkFiles(benchmarkPath):
        filePaths[files[0]] = files[1:]
        count += 1
print(count, "time series found")

## Run an algorithm and plot results

In [ ]:
@interact_manual(Algorithm=algorithms.keys(), TS_Files=sorted(filePaths.keys()))
def interactive_plot(Algorithm, TS_Files):
    tsDir = Path(TS_Files)
    tsPath = tsDir / filePaths[TS_Files][0]
    tsMetaPath = tsDir / filePaths[TS_Files][1]
    plotMotif(Algorithm, tsPath, tsMetaPath)

You can use the buttons below the figure to interactively zoom and move the plot.

The area at the specific indices is colored as follows:
* <span style="color:green">Green</span>: true positive
* <span style="color:olive">Yellow</span>: false negative
* <span style="color:red">Red</span>: false positive
* White: true negative

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>